In [ ]:
!pip install surprise

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy

In [ ]:
scores = pd.read_csv('scores.csv')

In [ ]:
scores['scores'] = np.float16(scores['scores'])

In [ ]:
movies = pd.read_csv('df_final.csv')

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(scores[['userId', 'movieId', 'scores']], reader)

In [ ]:
train_set, test_set = train_test_split(data, test_size=.2, random_state=21)

In [ ]:
model = SVD()

In [ ]:
model.fit(train_set)

In [ ]:
predictions = model.test(test_set)

In [ ]:
rmse = accuracy.rmse(predictions)

In [ ]:
rmse_test_means = []
factors = [1,2,3,5,8,13,21,34,55,89,144,233]

for factor in factors:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv= 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

In [ ]:
plt.scatter(factors, rmse_test_means)
plt.xlabel('Number of Factors')
plt.ylabel('RMSE Error')
plt.show()

In [ ]:
# Function to Evaluate and predict if the movie is recommender for the user or not.
def evaluate_recommendation_movie(userId, movieId):
    
    # Get the title of the introduced movie
    movie_title = movies[movies.id == movieId].title.iloc[0].title()

    # Evaluate the movie using the model
    prediction = model.predict(userId,str(movieId))

    if prediction.est > 3.6:
        return "Of course, get some popcorn and enjoy", prediction.est, movie_title
    else:
        return "No, you will sleep in the middle", prediction.est, movie_title

In [ ]:
!pip install gradio

In [ ]:
# --------------
# Interface Part
# --------------

# Import Gradio Library
import gradio as gr
    
title = str("Movie Recommendation System")

with gr.Blocks(title= title) as demo:
    with gr.Row():
        with gr.Column(scale=0.5, min_width=600):
            userId = gr.inputs.Number(label="Introduce your User ID")
            movieId = gr.Textbox(label="Introduce your Movie ID")
            evaluate_recommendation_movie_btn = gr.Button("Evaluate if the movie is for you")
            movie_title = gr.Textbox(label = "Film:")
            output = gr.Textbox(label="Will I enjoy the movie?")
            score = gr.Textbox(label="Predicted Score:")
            evaluate_recommendation_movie_btn.click(fn=evaluate_recommendation_movie, inputs=[userId,movieId], outputs=[output, score, movie_title])

demo.launch(share = True)